THESE commands are run in Powershell

Copy a File from Host to Docker Container

In [ ]:
docker cp
"E:\ModifiedDownloads\tripadvisor_european_restaurants\tripadvisor_european_rest
aurants.json" MONGO_CONTAINER:/tmp/tripadvisor_european_restaurants.json

Run an Interactive Bash Shell Inside a MongoDB Docker Container

In [ ]:
docker exec -it MONGO_CONTAINER bash

Import JSON Data into MongoDB

In [ ]:
mongoimport --db TripAdvisor --collection restaurants --file
/tmp/tripadvisor_european_restaurants.json –jsonArray

Connect to MongoDB Shell (`mongosh`)

In [ ]:
mongosh

Retrieve a Sample Document from the `restaurants` Collection

In [ ]:
db.restaurants.find().limit(1).pretty()

2.1 Find Top 5 Highly Rated Vegan-Friendly Breakfast Restaurants in Athens

In [ ]:
restaurants.find({
city: "Athens",
vegan_options: "Y",
meals: "Breakfast",
total_reviews_count: { $gt: 100 }
}, {
restaurant_name: 1,
address: 1,
avg_rating: 1,
total_reviews_count: 1,
_id: 0
}).sort({
avg_rating: -1,
total_reviews_count: -1
}).limit(5)

2.2 Find Top 10 Most Popular Cuisines in Vegan-Friendly Restaurants in Athens

In [ ]:
db.restaurants.aggregate([
{$match: {city: "Athens",vegan_options: "Y",}},
{$unwind: "$cuisines"},
{$group: {_id: "$cuisines",10total_restaurants: { $sum: 1 },avg_reviews: { $avg: "$total_reviews_count" }}},
{$sort: { total_restaurants: -1 }},
{$limit: 10}
])

2.3 Find the Top 5 Cities Outside Greece with the Most Greek Restaurants

In [ ]:
db.restaurants.aggregate([
{$match: {country: { $ne: "Greece" },cuisines: "Greek",}},
{$group: {_id: "$city",total_restaurants: { $sum: 1 }}},
{$sort: { total_restaurants: -1 }},
{$limit: 5}
])

2.4 Find the Top 5 Cities Outside Greece with the Most Greek Restaurants without Nulls

In [ ]:
db.restaurants.aggregate([
{$match: {country: { $ne: "Greece" },cuisines: "Greek",city: { $ne: null }}},
{$group: {_id: "$city",total_restaurants: { $sum: 1 }}},
{$sort: { total_restaurants: -1 }},
{$limit: 5}
])

2.5 Find the City with the Most Highly Rated Restaurants

In [ ]:
db.restaurants.aggregate([
{$match: {avg_rating: { $gt: 4.5 },total_reviews_count: { $gt: 1000 },city: { $ne: null }}},
{$group: {_id: "$city",total_restaurants: { $sum: 1 }}},
{$sort: { total_restaurants: -1 }},
{$limit: 1}
])

3.1 Optimizing Query Performance with Indexing in MongoDB

First we run the query for each question without any indexes and we get the statistics shown in the Report

After that we drop all previous Indexes we create the indexes for each query and rerun it

We compare statistics before and after indexing

This is the procedure followed for answering 3.1 of the project 

After that i just provide the indexes for each subquestion 

In [ ]:
restaurants.find({
city: "Athens",
vegan_options: "Y",
meals: "Breakfast",
total_reviews_count: { $gt: 100 }
}, {
restaurant_name: 1,
address: 1,
avg_rating: 1,
total_reviews_count: 1,
_id: 0
}).sort({
avg_rating: -1,
total_reviews_count: -1
}).limit(5).explain("executionStats")


In [ ]:
db.restaurants.createIndex({ city: 1, vegan_options: 1, meals: 1, total_reviews_count: 1 })

In [ ]:
restaurants.find({
city: "Athens",
vegan_options: "Y",
meals: "Breakfast",
total_reviews_count: { $gt: 100 }
}, {
restaurant_name: 1,
address: 1,
avg_rating: 1,
total_reviews_count: 1,
_id: 0
}).sort({
avg_rating: -1,
total_reviews_count: -1
}).limit(5).explain("executionStats")

In [ ]:
db.restaurants.dropIndexes()

3.2

In [ ]:
db.restaurants.createIndex({ city: 1, vegan_options: 1, cuisines: 1 })

3.3

In [ ]:
db.restaurants.createIndex({ country: 1, cuisines: 1, city: 1 })

3.4

In [ ]:
db.restaurants.createIndex({ avg_rating: 1, total_reviews_count: 1, city: 1 })

3.5

In [ ]:
db.restaurants.createIndex( { avg_rating: 1, total_reviews_count: 1, city: 1 } )